# 🏠 Ames Housing Data Cleaning

## ⚙️ 1. Setup & Constants


In [ ]:
import pandas as pd
from pathlib import Path

#constants
DATA_PATH = Path("../dataset/train.csv")
CLEANED_DATA_PATH = Path("../dataset/cleaned_train.csv")

## 📥 Load Raw Data

In [24]:
def load_raw_data():
    """Load raw data from CSV file."""
    df = pd.read_csv(DATA_PATH)
    print(f"Loaded raw data: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

df_raw = load_raw_data()
df = df_raw.copy()    

Loaded raw data: 1460 rows, 81 columns


## 🔍 Analyze Missing Values

In [25]:
def analyze_missing_values(df):
    """Analyze missing values in the DataFrame."""
    missing = df.isnull().sum()
    missing_percent = (missing/len(df)) * 100

    missing_df = pd.DataFrame({
        'missing_count' : missing,
        'missing_percent' : missing_percent,
        'missing_dtype' : df.dtypes
    })

    missing_df = missing_df[missing_df['missing_count']>0].sort_values(by='missing_percent', ascending=False)

    return missing_df

missing_values = analyze_missing_values(df)
missing_values.head(15)

,missing_count,missing_percent,missing_dtype
PoolQC,1453,99.520548,object
MiscFeature,1406,96.301370,object
Alley,1369,93.767123,object
Fence,1179,80.753425,object
MasVnrType,872,59.726027,object
FireplaceQu,690,47.260274,object
LotFrontage,259,17.739726,float64
GarageType,81,5.547945,object
GarageYrBlt,81,5.547945,float64
GarageFinish,81,5.547945,object


## 🧹 Handle Missing Values

In [26]:
def handle_missing_values(df):
    """Handle missing values in the DataFrame."""
    
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # LotFrontage → median per Neighborhood
    if "LotFrontage" in df.columns:
        df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
            lambda x: x.fillna(x.median())
        )
    
    # Numerical → median
    num_cols = df.select_dtypes(include=[np.number]).columns    
    for col in num_cols:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())

    # Absence categories → fill with 'NA'/'None'
    absence_fill = {
        "MasVnrType": "None","BsmtQual":"NA","BsmtCond":"NA","BsmtExposure":"NA",
        "BsmtFinType1":"NA","BsmtFinType2":"NA","GarageType":"NA","GarageFinish":"NA",
        "GarageQual":"NA","GarageCond":"NA","FireplaceQu":"NA","PoolQC":"NA","Fence":"NA",
        "Alley":"NA","MiscFeature":"NA"
    }
    
    for col, val in absence_fill.items():
        if col in df.columns:
            df[col] = df[col].fillna(val)

    # Electrical → mode
    if "Electrical" in df.columns and df['Electrical'].isnull().any():
        df["Electrical"] = df["Electrical"].fillna(df["Electrical"].mode()[0])  

    # Get all categorical columns as a set
    all_cat_cols = set(df.select_dtypes(include=["object"]).columns)
    
    # Remove already handled columns using set difference
    remaining_cats = all_cat_cols - set(absence_fill.keys()) - {"Electrical"}
    
    # Handle only the remaining categorical columns
    for col in remaining_cats:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    
    return df

## 🛠️ Fix Data Types

In [27]:
def fix_datatypes(df):
    df = df.copy()
    # Convert some columns to numeric if needed
    numeric_columns = ['LotFrontage','LotArea','MasVnrArea','GarageYrBlt']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    # Keep year columns as integers
    year_columns = ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold']
    for col in year_columns:
        if col in df.columns:
            df[col] = df[col].astype('Int64')
    return df

## ✂️ Handle Outliers

In [28]:
OUTLIER_COLUMNS = [
    'LotFrontage', 'LotArea', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'GrLivArea',
    'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'SalePrice'
]

COLUMN_THRESHOLDS = {
    'LotArea': 2.5, 'SalePrice': 2.5, 'GrLivArea': 2.5, 'default': 3
}

def handle_outliers(df, columns=OUTLIER_COLUMNS, thresholds=COLUMN_THRESHOLDS, 
                    method='clip', analyze=False, verbose=True):
    
    df = df.copy()
    outliers_info = {}

    for col in columns:
        if col not in df.columns:
            continue

        mean, std = df[col].mean(), df[col].std()
        if std == 0:
            continue
        n_std = thresholds.get(col, thresholds['default'])
        lower, upper = mean - n_std * std, mean + n_std * std

        mask = (df[col] < lower) | (df[col] > upper)
        outlier_count = mask.sum()

        # store analysis
        outliers_info[col] = {
            "count": int(outlier_count),
            "percent": round((outlier_count / len(df)) * 100, 2),
            "min": df[col].min(),
            "max": df[col].max(),
            "threshold_low": lower,
            "threshold_high": upper
        }

        # handle outliers
        if method == 'clip':
            df[col] = df[col].clip(lower, upper)
        elif method == 'remove':
            df = df.loc[~mask]

        if verbose and outlier_count > 0:
            print(f"{col}: {outlier_count} outliers → handled with '{method}'")

    analysis_df = pd.DataFrame(outliers_info).T

    return (df, analysis_df) if analyze else df


## 📈 Transform Skewed Features

In [29]:
def transform_skewed(df, columns, skew_threshold=1):
    df = df.copy()
    skewness = df[columns].apply(lambda x: x.skew())
    skewed_cols = skewness[skewness > skew_threshold].index.tolist()
    for col in skewed_cols:
        if (df[col] <= 0).any():
            continue  # avoid issues with log1p if non-positive values exist
        df[col] = np.log1p(df[col])
        print(f"{col}: skew {skewness[col]:.2f} → {df[col].skew():.2f}")
    return df, skewed_cols

## 💾 Save Cleaned Data

In [31]:
# Load raw data
df = load_raw_data()

# Clean step by step
df = handle_missing_values(df)
df = fix_datatypes(df)
df = handle_outliers(df, columns=OUTLIER_COLUMNS, thresholds=COLUMN_THRESHOLDS, method='clip', analyze=False, verbose=False)
df, transformed_cols = transform_skewed(df, OUTLIER_COLUMNS)

# Remove duplicates
dupes = df.duplicated().sum()
if dupes > 0:
    print(f"Dropped {dupes} duplicate rows")
    df = df.drop_duplicates()

# Save cleaned data
print(f"Final cleaned data: {df.shape[0]} rows, {df.shape[1]} columns")
df.to_csv(CLEANED_DATA_PATH, index=False)
print(f"✅ Cleaned dataset saved to {CLEANED_DATA_PATH}")

Loaded raw data: 1460 rows, 81 columns
LotArea: skew 2.25 → -0.80
SalePrice: skew 1.01 → -0.08
Final cleaned data: 1460 rows, 81 columns
✅ Cleaned dataset saved to ..\dataset\cleaned_train.csv


---
## ✅ Cleaning Summary
- Missing values handled (LotFrontage, categorical NA, numerical → median, Electrical → mode).
- Data types fixed (years kept as integers).
- Outliers clipped using z-scores with custom thresholds.
- Skewed numeric features transformed (`log1p`).
- Duplicates removed.
- Final cleaned dataset exported for modeling.
